<h1 align="center">
<img src="https://drive.google.com/uc?export=view&id=1_ZkE3dnYg9KykzdZE6lIK2s_q_nbiKk_" width="160" height="160">
</h1>
<h1 align="center">Massive Data Analytics - 25577</h1>
<h3 align="center">Dr. Gholampour</h3>
<h5 align="center">Sharif University of Technology<br/>Electrical Engineering Department<br/>  Fall 2022</h5>

---

# Problem Set 02
<h4 align="center">Vahid Pourakbar     400205511</h4>

# Question 3

## Part 1

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("MDA_2022_PS2_Q3") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

22/12/20 18:04:04 WARN Utils: Your hostname, vahid-PC resolves to a loopback address: 127.0.1.1; using 172.27.210.205 instead (on interface wlp2s0)
22/12/20 18:04:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/20 18:04:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
records = sc.textFile("records.txt")
records.take(2)

['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ',
 'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ']

In [4]:
baskets = records.map(lambda x: x.strip().split(' '))
baskets.take(2)

[['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222'],
 ['GRO99222',
  'GRO12298',
  'FRO12685',
  'ELE91550',
  'SNA11465',
  'ELE26917',
  'ELE52966',
  'FRO90334',
  'SNA30755',
  'ELE17451',
  'FRO84225',
  'SNA80192']]

In [5]:
baskets_without_duplicates = baskets.map(lambda x: list(set(x)))
baskets_without_duplicates.take(2)

[['ELE17451', 'FRO11987', 'SNA90258', 'ELE89019', 'GRO99222'],
 ['ELE17451',
  'SNA11465',
  'FRO90334',
  'SNA80192',
  'SNA30755',
  'FRO12685',
  'GRO12298',
  'ELE91550',
  'FRO84225',
  'ELE26917',
  'ELE52966',
  'GRO99222']]

In [6]:
items = baskets_without_duplicates.flatMap(lambda x:x)
items.take(15)

['ELE17451',
 'FRO11987',
 'SNA90258',
 'ELE89019',
 'GRO99222',
 'ELE17451',
 'SNA11465',
 'FRO90334',
 'SNA80192',
 'SNA30755',
 'FRO12685',
 'GRO12298',
 'ELE91550',
 'FRO84225',
 'ELE26917']

In [7]:
item_counts = items.map(lambda x: (x, 1)) \
    .reduceByKey(lambda x,y: x+y)
item_counts.take(10)

[('FRO11987', 104),
 ('SNA90258', 550),
 ('FRO90334', 63),
 ('SNA80192', 258),
 ('ELE91550', 23),
 ('FRO84225', 74),
 ('ELE52966', 380),
 ('GRO73461', 3602),
 ('DAI22896', 1219),
 ('FRO86643', 235)]

In [8]:
support_threshold = 100
frequent_items = item_counts.filter(lambda x: x[1] >= support_threshold)
frequent_items.take(10)

[('FRO11987', 104),
 ('SNA90258', 550),
 ('SNA80192', 258),
 ('ELE52966', 380),
 ('GRO73461', 3602),
 ('DAI22896', 1219),
 ('FRO86643', 235),
 ('SNA69641', 599),
 ('GRO39357', 296),
 ('ELE11375', 214)]

In [9]:
frequent_item_list = frequent_items.map(lambda x: x[0]).collect()
frequent_item_list[:10]

['FRO11987',
 'SNA90258',
 'SNA80192',
 'ELE52966',
 'GRO73461',
 'DAI22896',
 'FRO86643',
 'SNA69641',
 'GRO39357',
 'ELE11375']

In [10]:
# remove non-frequent items in each basket
basket_of_frequent_items = baskets_without_duplicates.map(lambda x: [i for i in x if i in frequent_item_list])
basket_of_frequent_items.take(2)

[['ELE17451', 'FRO11987', 'SNA90258', 'GRO99222'],
 ['ELE17451',
  'SNA11465',
  'SNA80192',
  'SNA30755',
  'GRO12298',
  'ELE26917',
  'ELE52966',
  'GRO99222']]

## k = 2

In [11]:
paired_frequent_items_list = basket_of_frequent_items.map(lambda x: sorted(x)) \
    .flatMap(lambda x: [(a, b) for idx, a in enumerate(x) for b in x[idx + 1:]])
paired_frequent_items_list.take(10)

[('ELE17451', 'FRO11987'),
 ('ELE17451', 'GRO99222'),
 ('ELE17451', 'SNA90258'),
 ('FRO11987', 'GRO99222'),
 ('FRO11987', 'SNA90258'),
 ('GRO99222', 'SNA90258'),
 ('ELE17451', 'ELE26917'),
 ('ELE17451', 'ELE52966'),
 ('ELE17451', 'GRO12298'),
 ('ELE17451', 'GRO99222')]

In [12]:
paired_frequent_items_count = paired_frequent_items_list.map(lambda x: (x,1)) \
    .reduceByKey(lambda x, y: x+y)
paired_frequent_items_count.take(10)

[(('ELE17451', 'GRO99222'), 148),
 (('FRO11987', 'SNA90258'), 2),
 (('ELE17451', 'ELE26917'), 314),
 (('ELE17451', 'GRO12298'), 36),
 (('ELE17451', 'SNA11465'), 17),
 (('ELE17451', 'SNA30755'), 111),
 (('ELE26917', 'GRO12298'), 17),
 (('ELE26917', 'GRO99222'), 192),
 (('ELE26917', 'SNA11465'), 20),
 (('ELE26917', 'SNA30755'), 67)]

In [13]:
frequent_pairs = paired_frequent_items_count.filter(lambda x: x[1]>= support_threshold)
frequent_pairs.take(10)

[(('ELE17451', 'GRO99222'), 148),
 (('ELE17451', 'ELE26917'), 314),
 (('ELE17451', 'SNA30755'), 111),
 (('ELE26917', 'GRO99222'), 192),
 (('DAI22896', 'GRO73461'), 304),
 (('ELE17451', 'SNA99873'), 270),
 (('ELE17451', 'GRO56989'), 129),
 (('ELE17451', 'FRO78087'), 218),
 (('DAI22177', 'ELE66810'), 105),
 (('DAI22177', 'GRO73461'), 248)]

In [55]:
frequent_items_count = dict(frequent_items.collect())

def Confidence_Score(pair):
  result = []
  X, Y, count = pair[0][0], pair[0][1], pair[1]
  # X->Y, Y->X
  result.append(((X, Y), count/frequent_items_count[X]))
  result.append(((Y, X), count/frequent_items_count[Y]))
  return result

confidencescore = frequent_pairs.flatMap(lambda pairCount: Confidence_Score(pairCount)) \
          .sortBy(lambda a: -a[1])
confidencescore.take(10)

[(('DAI93865', 'FRO40251'), 1.0),
 (('GRO85051', 'FRO40251'), 0.999176276771005),
 (('GRO38636', 'FRO40251'), 0.9906542056074766),
 (('ELE12951', 'FRO40251'), 0.9905660377358491),
 (('DAI88079', 'FRO40251'), 0.9867256637168141),
 (('FRO92469', 'FRO40251'), 0.983510011778563),
 (('DAI43868', 'SNA82528'), 0.972972972972973),
 (('DAI23334', 'DAI62779'), 0.9545454545454546),
 (('ELE92920', 'DAI62779'), 0.7326649958228906),
 (('DAI53152', 'FRO40251'), 0.717948717948718)]

## k = 3

In [20]:
candidate_list = set(frequent_pairs.flatMap(lambda x: x[0]).collect())

{'DAI16732',
 'DAI18527',
 'DAI22177',
 'DAI22240',
 'DAI22534',
 'DAI22896',
 'DAI23334',
 'DAI29159',
 'DAI31081',
 'DAI33336',
 'DAI35347',
 'DAI37288',
 'DAI38969',
 'DAI42083',
 'DAI42493',
 'DAI43223',
 'DAI43868',
 'DAI46755',
 'DAI48126',
 'DAI48891',
 'DAI49199',
 'DAI50913',
 'DAI50921',
 'DAI51880',
 'DAI53152',
 'DAI55148',
 'DAI55911',
 'DAI59508',
 'DAI62779',
 'DAI63921',
 'DAI67189',
 'DAI67621',
 'DAI70456',
 'DAI73122',
 'DAI75645',
 'DAI83031',
 'DAI83733',
 'DAI83948',
 'DAI85309',
 'DAI86167',
 'DAI87448',
 'DAI88079',
 'DAI88088',
 'DAI88352',
 'DAI88807',
 'DAI91290',
 'DAI92600',
 'DAI93865',
 'DAI94679',
 'DAI95741',
 'ELE11111',
 'ELE11160',
 'ELE12792',
 'ELE12845',
 'ELE12951',
 'ELE14480',
 'ELE17451',
 'ELE20196',
 'ELE20398',
 'ELE20847',
 'ELE21353',
 'ELE24630',
 'ELE25077',
 'ELE26917',
 'ELE28189',
 'ELE30182',
 'ELE30933',
 'ELE32164',
 'ELE32244',
 'ELE34057',
 'ELE36909',
 'ELE37048',
 'ELE37770',
 'ELE37798',
 'ELE38289',
 'ELE49801',
 'ELE53126',

In [23]:
baskets_of_paired_frequent_items = baskets_without_duplicates.map(lambda x: [i for i in x if i in candidate_list])
baskets_of_paired_frequent_items.take(2)

[['ELE17451', 'SNA90258', 'GRO99222'],
 ['ELE17451', 'SNA30755', 'ELE26917', 'GRO99222']]

In [26]:
triple_frequent_items_list = baskets_of_paired_frequent_items.map(lambda x: sorted(x)) \
    .flatMap(lambda x: [(a, b, c) for idx, a in enumerate(x) for idx2, b in enumerate(x[idx + 1:]) for c in x[idx+idx2+2:]])
triple_frequent_items_list.take(10)

[('ELE17451', 'GRO99222', 'SNA90258'),
 ('ELE17451', 'ELE26917', 'GRO99222'),
 ('ELE17451', 'ELE26917', 'SNA30755'),
 ('ELE17451', 'GRO99222', 'SNA30755'),
 ('ELE26917', 'GRO99222', 'SNA30755'),
 ('DAI22896', 'ELE17451', 'GRO73461'),
 ('DAI22896', 'ELE17451', 'SNA99873'),
 ('DAI22896', 'GRO73461', 'SNA99873'),
 ('ELE17451', 'GRO73461', 'SNA99873'),
 ('ELE17451', 'ELE37798', 'GRO56989')]

In [27]:
triple_frequent_items_count = triple_frequent_items_list.map(lambda x: (x,1)) \
    .reduceByKey(lambda x, y: x+y)
triple_frequent_items_count.take(10)

[(('ELE17451', 'ELE26917', 'GRO99222'), 32),
 (('ELE17451', 'ELE26917', 'SNA30755'), 22),
 (('ELE17451', 'GRO99222', 'SNA30755'), 4),
 (('ELE26917', 'GRO99222', 'SNA30755'), 2),
 (('DAI22896', 'ELE17451', 'GRO73461'), 48),
 (('DAI22896', 'GRO73461', 'SNA99873'), 28),
 (('ELE17451', 'ELE37798', 'GRO56989'), 8),
 (('ELE17451', 'GRO39357', 'SNA69641'), 1),
 (('FRO78087', 'GRO39357', 'SNA69641'), 2),
 (('DAI22896', 'DAI50921', 'ELE17451'), 3)]

In [29]:
frequent_triple_pairs = triple_frequent_items_count.filter(lambda x: x[1]>= support_threshold)
frequent_triple_pairs.take(10)

[(('ELE17451', 'SNA59903', 'SNA72163'), 127),
 (('DAI62779', 'ELE17451', 'FRO78087'), 121),
 (('DAI62779', 'ELE17451', 'ELE26917'), 160),
 (('DAI62779', 'ELE17451', 'SNA55762'), 157),
 (('DAI62779', 'ELE17451', 'SNA99873'), 126),
 (('DAI62779', 'ELE17451', 'SNA45677'), 158),
 (('DAI62779', 'ELE17451', 'FRO40251'), 406),
 (('DAI62779', 'FRO31317', 'GRO73461'), 100),
 (('DAI62779', 'ELE17451', 'GRO30386'), 218),
 (('DAI62779', 'ELE32164', 'GRO73461'), 131)]

In [54]:
frequent_pairs_count = dict(frequent_pairs.collect())

def Confidence_Score_3(pair):
  result = []
  X, Y, Z, count = pair[0][0], pair[0][1], pair[0][2], pair[1]
  
  result.append(((X, Y, Z), count/frequent_pairs_count[(X, Y)]))
  result.append(((X, Z, Y), count/frequent_pairs_count[(X, Z)]))
  result.append(((Y, Z, X), count/frequent_pairs_count[(Y, Z)]))
  return result

confidencescore_3 = frequent_triple_pairs.flatMap(lambda pairCount: Confidence_Score_3(pairCount)) \
        .sortBy(lambda a: -a[1])
confidencescore_3.take(30)

[(('GRO73461', 'GRO85051', 'FRO40251'), 1.0),
 (('GRO85051', 'SNA80324', 'FRO40251'), 1.0),
 (('DAI75645', 'GRO85051', 'FRO40251'), 1.0),
 (('DAI55911', 'GRO85051', 'FRO40251'), 1.0),
 (('ELE20847', 'GRO85051', 'FRO40251'), 1.0),
 (('DAI23334', 'ELE92920', 'DAI62779'), 1.0),
 (('DAI62779', 'DAI88079', 'FRO40251'), 1.0),
 (('ELE17451', 'GRO85051', 'FRO40251'), 1.0),
 (('ELE26917', 'GRO85051', 'FRO40251'), 1.0),
 (('GRO38814', 'GRO85051', 'FRO40251'), 1.0),
 (('GRO85051', 'SNA45677', 'FRO40251'), 1.0),
 (('FRO53271', 'GRO85051', 'FRO40251'), 1.0),
 (('DAI31081', 'GRO85051', 'FRO40251'), 1.0),
 (('ELE20847', 'FRO92469', 'FRO40251'), 1.0),
 (('GRO21487', 'GRO85051', 'FRO40251'), 1.0),
 (('DAI62779', 'GRO85051', 'FRO40251'), 0.9973821989528796),
 (('DAI75645', 'DAI88079', 'FRO40251'), 0.9932885906040269),
 (('DAI88079', 'GRO73461', 'FRO40251'), 0.993103448275862),
 (('DAI88079', 'ELE17451', 'FRO40251'), 0.9919354838709677),
 (('FRO92469', 'GRO73461', 'FRO40251'), 0.9906103286384976),
 (('DA